In [79]:
from sympy import *
from sympy import sin, pde_separate
from sympy.physics.vector import dynamicsymbols
from sympy.physics.vector.printing import vlatex, vpprint

# import numpy as np
import matplotlib.pyplot as plt
init_printing(use_latex='mathjax',use_unicode=True)

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))

In [38]:
q1, q2, q3, q4, q5, q6 = dynamicsymbols('q1 q2 q3 q4 q5 q6')
dq1, dq2, dq3, dq4, dq5, dq6 = symbols('dq1 dq2 dq3 dq4 dq5 dq6')
ddq1, ddq2, ddq3, ddq4, ddq5, ddq6 = symbols('ddq1 ddq2 ddq3 ddq4 ddq5 ddq6')
Q1, Q2, Q3, Q4, Q5, Q6 = dynamicsymbols('Q1 Q2 Q3 Q4 Q5 Q6')
m, Ixx, Iyy, Izz, g = symbols('m Ixx Iyy Izz g ', constant = True)

q_3, q_5, q_6 = symbols('q_3 q_5 q_6')

In [3]:
q = Matrix([q1, q2, q3, q4, q5, q6])
dq = diff(q, Symbol('t'))
ddq = diff(dq, Symbol('t'))
# dq = Matrix([dq1, dq2, dq3, dq4, dq5, dq6])
# ddq

$
T(q)=\frac{1}{2}\dot{q}^{T}M(q)*\dot{q}
$

where M(q) is defined as:

$
M(q)=J_{v}(q)^{T}m_{rr}J_{v}(q)+J_{w}(q)^{T}m_{\theta\theta}J_{w}(q)
$

In [4]:
J_w = Matrix([[0,0,0,-sin(q[4]), 0, 1],
             [0,0,0, cos(q[4])*sin(q[5]), cos(q[5]), 0],
             [0,0,0, cos(q[4])*cos(q[5]), sin(q[5]), 0]])
J_v = Matrix([[1,0,0,0,0,0],
             [0,1,0,0,0,0],
             [0,0,1,0,0,0]])
m_rr = eye(3)*m
m_tt = Matrix([[Ixx, 0, 0],
              [0, Iyy, 0],
              [0, 0, Izz]])
M_q = J_v.T * m_rr * J_v + J_w.T * m_tt * J_w
M_q = trigsimp(M_q)

# T = trigsimp(trigsimp(expand(0.5*dq.T*M_q*dq)))
# T = trigsimp(expand(0.5*dq.T*M_q*dq))
T = trigsimp(simplify(trigsimp(0.5*dq.T*M_q*dq)))

V = -m*g*q[2]

L = T[0]-V
L = collect(L, [dq[3]**2, dq[4]**2])

L = L.trigsimp(method = 'fu')
L

                   2                                2                    2    
        ⎛d        ⎞                      ⎛d        ⎞          ⎛d        ⎞     
0.5⋅Ixx⋅⎜──(q₆(t))⎟  + g⋅m⋅q₃(t) + 0.5⋅m⋅⎜──(q₁(t))⎟  + 0.5⋅m⋅⎜──(q₂(t))⎟  + 0
        ⎝dt       ⎠                      ⎝dt       ⎠          ⎝dt       ⎠     

                2                                                             
     ⎛d        ⎞    ⎛           2                     2                 ⎞ ⎛d  
.5⋅m⋅⎜──(q₃(t))⎟  + ⎝0.5⋅Iyy⋅cos (q₆(t)) - 0.5⋅Izz⋅cos (q₆(t)) + 0.5⋅Izz⎠⋅⎜──(
     ⎝dt       ⎠                                                          ⎝dt 

       2                                                                      
      ⎞    ⎛                     d                                            
q₅(t))⎟  + ⎜- 1.0⋅Ixx⋅sin(q₅(t))⋅──(q₆(t)) + 0.25⋅Iyy⋅(-sin(q₅(t) - 2⋅q₆(t)) +
      ⎠    ⎝                     dt                                           

                                                 

The L above is with no trig simplification, actually it is expanded.

In the next cell you are looking at L but simplified with trig by hand.

In [5]:
L = (0.5*m*dq[0]**2 + 
     0.5*m*dq[1]**2 + 
     0.5*m*dq[2]**2 + 
     (0.5*Ixx*sin(q[4])**2 + 0.5*Iyy*cos(q[5])**2 - 0.5*(Iyy-Izz)*(cos(q[4])**2)*cos(q[5])**2)*dq[3]**2 +
     0.5*((Iyy - Izz)*cos(q[5])**2 + Izz)*dq[4]**2 + 
     0.5*Ixx*dq[5]**2 +
     (Iyy + Izz)*cos(q[4])*sin(q[5])*cos(q[5])*dq[3]*dq[4] - 
     Ixx*sin(q[4])*dq[3]*dq[5] +
     m*g*q[2]
    )

L

                                                          2                   
                 d         d                   ⎛d        ⎞                    
- Ixx⋅sin(q₅(t))⋅──(q₄(t))⋅──(q₆(t)) + 0.5⋅Ixx⋅⎜──(q₆(t))⎟  + g⋅m⋅q₃(t) + 0.5⋅
                 dt        dt                  ⎝dt       ⎠                    

             2                    2                    2                      
  ⎛d        ⎞          ⎛d        ⎞          ⎛d        ⎞                       
m⋅⎜──(q₁(t))⎟  + 0.5⋅m⋅⎜──(q₂(t))⎟  + 0.5⋅m⋅⎜──(q₃(t))⎟  + (Iyy + Izz)⋅sin(q₆(
  ⎝dt       ⎠          ⎝dt       ⎠          ⎝dt       ⎠                       

                                                                              
                          d         d           ⎛                             
t))⋅cos(q₅(t))⋅cos(q₆(t))⋅──(q₄(t))⋅──(q₅(t)) + ⎝0.5⋅Izz + 0.5⋅(Iyy - Izz)⋅cos
                          dt        dt                                        

                     2                           

Euler-Lagrange equations of motions from the following equation:

$
\frac{d}{dt}(\frac{\partial\mathcal{L}(q)}{\partial\dot{q_{j}}})-\frac{\partial\mathcal{L}(q)}{\partial q_{j}}=Q_{j}(q)
$


In [73]:
# d/dt(dL/d(dq)) - dL/dq = Q
dl1 = diff(diff(L, dq[0]), Symbol('t'))
dl2 = diff(diff(L, dq[1]), Symbol('t'))
dl3 = diff(diff(L, dq[2]), Symbol('t'))
dl4 = diff(diff(L, dq[3]), Symbol('t'))
dl5 = diff(diff(L, dq[4]), Symbol('t'))
dl6 = diff(diff(L, dq[5]), Symbol('t'))

l1 = diff(L, q[0])
l2 = diff(L, q[1])
l3 = diff(L, q[2])
l4 = diff(L, q[3])
l5 = diff(L, q[4])
l6 = diff(L, q[5])

EL1 = dl1 - l1 - Q1
EL2 = dl2 - l2 - Q2
EL3 = dl3 - l3 - Q3
EL4 = dl4 - l4 - Q4
EL5 = dl5 - l5 - Q5
EL6 = dl6 - l6 - Q6

# Q = Matrix([Q1, Q2, Q3, Q4, Q5, Q6])
# print(vlatex(EL4))
    
# EL1.coeff(ddq[0])

EL6.coeff(ddq[5])

eqs = (EL1, EL2, EL3, EL4, EL5, EL6)
variables = (ddq[0], ddq[1], ddq[2], ddq[3], ddq[4], ddq[5])
EL4

                   2                                                          
                  d                          d         d                      
- Ixx⋅sin(q₅(t))⋅───(q₆(t)) - Ixx⋅cos(q₅(t))⋅──(q₅(t))⋅──(q₆(t)) - (Iyy + Izz)
                   2                         dt        dt                     
                 dt                                                           

                                             2                                
                                  ⎛d        ⎞                   2             
⋅sin(q₅(t))⋅sin(q₆(t))⋅cos(q₆(t))⋅⎜──(q₅(t))⎟  - (Iyy + Izz)⋅sin (q₆(t))⋅cos(q
                                  ⎝dt       ⎠                                 
                                                                              

                                                                           2  
      d         d                                                         d   
₅(t))⋅──(q₅(t))⋅──(q₆(t)) + (Iyy + Izz)⋅sin(q₆(t))

In [74]:
EL4 = (-(0.5*Iyy - 0.5*Izz)*(cos(q[4])**2)*(cos(q[5])**2)*ddq[3] +
       (Iyy + Izz)*sin(q[5])*cos(q[4])*cos(q[5])*ddq[4] -
       Ixx*sin(q[4])*ddq[5] +
       2*sin(q[4])*cos(q[4])*(Ixx + (Iyy - Izz)*cos(q[5])**2)*dq[3]*dq[4] +
       sin(q[5])*cos(q[5])*((Iyy-Izz)*cos(q[4])**2 - Ixx)*dq[3]*dq[5] -
       Q4)

EL5 = ((Iyy + Izz)*sin(q[5])*cos(q[4])*cos(q[5])*ddq[3] +
       (Izz + (Iyy - Izz)*cos(q[5])**2)*ddq[4] +
       sin(q[4])*cos(q[4])*(Ixx + (Iyy - Izz)*cos(q[5])**2)*dq[3]**2 -
       2*(Iyy - Izz)*sin(q[5])*cos(q[5])*dq[4]*dq[5] +
       cos(q[4])*(Ixx +(Iyy + Izz)*cos(2*q[5]))*dq[3]*dq[5] -
       Q5)

EL6 = (-Ixx*sin(q[4])*ddq[3] +
       Ixx*ddq[5] -
       sin(q[5])*cos(q[5])*((Iyy-Izz)*cos(q[4])**2 + Iyy)*dq[3]**2 +
       (Iyy-Izz)*sin(q[5])*cos(q[5])*dq[4]**2 -
       cos(q[4])*(Ixx + (Iyy + Izz)*cos(2*q[5]))*dq[3]*dq[4] -
       Q6)

# EL4 = (-(0.5*Iyy - 0.5*Izz)*(cos(q_5)**2)*(cos(q_6)**2)*ddq4 +
#        (Iyy + Izz)*sin(q_6)*cos(q_5)*cos(q_6)*ddq5 -
#        Ixx*sin(q_5)*ddq6 +
#        2*sin(q_5)*cos(q_5)*(Ixx + (Iyy - Izz)*cos(q_6)**2)*dq4*dq5 +
#        sin(q_6)*cos(q_6)*((Iyy-Izz)*cos(q_5)**2 - Ixx)*dq4*dq6 -
#        Q4)


# EL5 = ((Iyy + Izz)*sin(q_6)*cos(q_5)*cos(q_6)*ddq4 +
#        (Izz + (Iyy - Izz)*cos(q_6)**2)*ddq5 +
#        sin(q_5)*cos(q_5)*(Ixx + (Iyy - Izz)*cos(q_6)**2)*dq4**2 -
#        2*(Iyy - Izz)*sin(q_6)*cos(q_6)*dq5*dq6 +
#        cos(q_5)*(Ixx +(Iyy + Izz)*cos(2*q_6))*dq4*dq6 -
#        Q5)



# EL6 = (-Ixx*sin(q_5)*ddq4 +
#        Ixx*ddq6 -
#        sin(q_6)*cos(q_6)*((Iyy-Izz)*cos(q_5)**2 + Iyy)*dq4**2 +
#        (Iyy-Izz)*sin(q_6)*cos(q_6)*dq5**2 -
#        cos(q_5)*(Ixx + (Iyy + Izz)*cos(2*q_6))*dq4*dq5 -
#        Q6)

eqs = (EL1, EL2, EL3, EL4, EL5, EL6)
variables = (ddq[0], ddq[1], ddq[2], ddq[3], ddq[4], ddq[5])
EL6

                   2                2                                         
                  d                d                                          
- Ixx⋅sin(q₅(t))⋅───(q₄(t)) + Ixx⋅───(q₆(t)) - (Ixx + (Iyy + Izz)⋅cos(2⋅q₆(t))
                   2                2                                         
                 dt               dt                                          

                                                                              
             d         d                                             ⎛d       
)⋅cos(q₅(t))⋅──(q₄(t))⋅──(q₅(t)) + (Iyy - Izz)⋅sin(q₆(t))⋅cos(q₆(t))⋅⎜──(q₅(t)
             dt        dt                                            ⎝dt      
                                                                              

  2                                                                    2      
 ⎞    ⎛                     2       ⎞                       ⎛d        ⎞       
)⎟  - ⎝Iyy + (Iyy - Izz)⋅cos (q₅(t))⎠⋅sin(q₆(t))⋅c

In [75]:
# eqs = (EL1, EL2, EL3, EL4, EL5, EL6)
# variables = (ddq[0], ddq[1], ddq[2], ddq4, ddq5, ddq6)

F = solve(eqs, variables)

In [83]:
F1 = F[ddq[0]]
F2 = F[ddq[1]]
F3 = F[ddq[2]]
# F4 = F[ddq4]
# F5 = F[ddq5]
# F6 = F[ddq6]

F4 = F[ddq[3]]
F5 = F[ddq[4]]
F6 = F[ddq[5]]

F3



    Q₃(t)
g + ─────
      m  

In [87]:
A = (sympify(F4, evaluate=False)).args

In [109]:
B = (sympify(A[2], evaluate=False)).args
B[2]

                ⎛                                  2                          
                ⎜                       ⎛d        ⎞                   d       
2.0⋅(Iyy + Izz)⋅⎜0.5⋅Ixx⋅sin(2.0⋅q₅(t))⋅⎜──(q₄(t))⎟  + Ixx⋅cos(q₅(t))⋅──(q₄(t)
                ⎝                       ⎝dt       ⎠                   dt      

                                                               2              
  d                                        2        ⎛d        ⎞               
)⋅──(q₆(t)) + Iyy⋅sin(q₅(t))⋅cos(q₅(t))⋅cos (q₆(t))⋅⎜──(q₄(t))⎟  - Iyy⋅sin(2.0
  dt                                                ⎝dt       ⎠               

                                                                              
        d         d                                         d         d       
⋅q₆(t))⋅──(q₅(t))⋅──(q₆(t)) + Iyy⋅cos(q₅(t))⋅cos(2.0⋅q₆(t))⋅──(q₄(t))⋅──(q₆(t)
        dt        dt                                        dt        dt      

                                                 

$
\frac{d\dot{q_{4}}}{dt} = \frac{1.0 \left(\left(0.25 Iyy + 0.25 Izz\right) \left(- 1.0 Ixx \operatorname{sin}\left(2.0 q_{5}\right) \dot{q}_{4}^{2} + 2.0 Ixx \operatorname{cos}\left(q_{5}\right) \dot{q}_{4} \dot{q}_{6} - 0.5 Iyy \operatorname{sin}\left(2.0 q_{5}\right) \operatorname{cos}\left(2.0 q_{6}\right) \dot{q}_{4}^{2} + 0.5 Iyy \operatorname{sin}\left(2.0 q_{5}\right) \dot{q}_{4}^{2} - 2.0 Iyy \operatorname{sin}\left(2.0 q_{6}\right) \dot{q}_{5} \dot{q}_{6} + 2.0 Iyy \operatorname{cos}\left(q_{5}\right) \operatorname{cos}\left(2.0 q_{6}\right) \dot{q}_{4} \dot{q}_{6} + 0.5 Izz \operatorname{sin}\left(2.0 q_{5}\right) \operatorname{cos}\left(2.0 q_{6}\right) \dot{q}_{4}^{2} + 0.5 Izz \operatorname{sin}\left(2.0 q_{5}\right) \dot{q}_{4}^{2} + 2.0 Izz \operatorname{sin}\left(2.0 q_{6}\right) \dot{q}_{5} \dot{q}_{6} + 2.0 Izz \operatorname{cos}\left(q_{5}\right) \operatorname{cos}\left(2.0 q_{6}\right) \dot{q}_{4} \dot{q}_{6} - 2.0 Q_{5}\right) \operatorname{sin}\left(2.0 q_{6}\right) \operatorname{cos}\left(q_{5}\right) + 0.5 \left(Iyy \operatorname{cos}^{2}\left(q_{6}\right) - Izz \operatorname{cos}^{2}\left(q_{6}\right) + Izz\right) \left(2.0 Ixx \operatorname{cos}\left(q_{5}\right) \dot{q}_{4} \dot{q}_{5} + 2.0 Iyy \operatorname{sin}\left(q_{6}\right) \operatorname{cos}^{2}\left(q_{5}\right) \operatorname{cos}\left(q_{6}\right) \dot{q}_{4}^{2} - Iyy \operatorname{sin}\left(2.0 q_{6}\right) \dot{q}_{5}^{2} + Iyy \operatorname{cos}\left(q_{5} - 2.0 q_{6}\right) \dot{q}_{4} \dot{q}_{5} + Iyy \operatorname{cos}\left(q_{5} + 2.0 q_{6}\right) \dot{q}_{4} \dot{q}_{5} - 2.0 Izz \operatorname{sin}\left(q_{6}\right) \operatorname{cos}^{2}\left(q_{5}\right) \operatorname{cos}\left(q_{6}\right) \dot{q}_{4}^{2} + Izz \operatorname{sin}\left(2.0 q_{6}\right) \dot{q}_{5}^{2} + Izz \operatorname{cos}\left(q_{5} - 2.0 q_{6}\right) \dot{q}_{4} \dot{q}_{5} + Izz \operatorname{cos}\left(q_{5} + 2.0 q_{6}\right) \dot{q}_{4} \dot{q}_{5} + 2.0 Q_{6}\right) \operatorname{sin}\left(q_{5}\right) - 0.25 \left(Iyy \operatorname{cos}^{2}\left(q_{6}\right) - Izz \operatorname{cos}^{2}\left(q_{6}\right) + Izz\right) \left(4.0 Ixx \operatorname{sin}\left(2.0 q_{5}\right) \dot{q}_{4} \dot{q}_{5} - 4.0 Ixx \operatorname{cos}\left(q_{5}\right) \dot{q}_{5} \dot{q}_{6} - 2.0 Iyy \operatorname{sin}\left(q_{5}\right) \operatorname{sin}\left(2.0 q_{6}\right) \dot{q}_{5}^{2} + 2.0 Iyy \operatorname{sin}\left(2.0 q_{5}\right) \operatorname{cos}\left(2.0 q_{6}\right) \dot{q}_{4} \dot{q}_{5} - 2.0 Iyy \operatorname{sin}\left(2.0 q_{5}\right) \dot{q}_{4} \dot{q}_{5} + 2.0 Iyy \operatorname{sin}\left(2.0 q_{6}\right) \operatorname{cos}\left(2.0 q_{5}\right) \dot{q}_{4} \dot{q}_{6} + 2.0 Iyy \operatorname{sin}\left(2.0 q_{6}\right) \dot{q}_{4} \dot{q}_{6} + 4.0 Iyy \operatorname{cos}\left(q_{5}\right) \operatorname{cos}\left(2.0 q_{6}\right) \dot{q}_{5} \dot{q}_{6} - 2.0 Izz \operatorname{sin}\left(q_{5}\right) \operatorname{sin}\left(2.0 q_{6}\right) \dot{q}_{5}^{2} - 2.0 Izz \operatorname{sin}\left(2.0 q_{5}\right) \operatorname{cos}\left(2.0 q_{6}\right) \dot{q}_{4} \dot{q}_{5} - 2.0 Izz \operatorname{sin}\left(2.0 q_{5}\right) \dot{q}_{4} \dot{q}_{5} - 2.0 Izz \operatorname{sin}\left(2.0 q_{6}\right) \operatorname{cos}\left(2.0 q_{5}\right) \dot{q}_{4} \dot{q}_{6} - 2.0 Izz \operatorname{sin}\left(2.0 q_{6}\right) \dot{q}_{4} \dot{q}_{6} + 4.0 Izz \operatorname{cos}\left(q_{5}\right) \operatorname{cos}\left(2.0 q_{6}\right) \dot{q}_{5} \dot{q}_{6} - 4.0 Q_{4}\right)\right)}{Iyy Izz \operatorname{cos}^{2}\left(q_{5}\right) \operatorname{cos}^{2}\left(2 q_{6}\right)}
$